In [ ]:
# 22/09/2022
# Author: Andrea Mendonça Inácio

# WebScraping do Portal Animal da cidade de Campinas
# Departamento de Proteção e Bem Estar Animal - Prefeitura de Campinas
# https://portalanimal.campinas.sp.gov.br/adocao

# Animais (principalmente cachorros e gatos) resgatados de situação de rua, machucados
# atropelados, ou em outras situações. Disponíveis para adoção.

# O objetivo é raspar o Portal Animal e exportar todos os animais disponíveis para
# adoção em um banco de dados. A partir daí extrair algumas Queries que interessam a
# quem procura um determinado tipo de cachorro ou gato para adoção, de forma mais rápida 
# e assertiva.


In [ ]:
import pandas as pd
import sqlite3
import requests as rq
import re
import json
from bs4 import BeautifulSoup
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
db_doacoes_df = pd.DataFrame (columns=['nome','especie','sexo','idade','porte','extra_info'])
db_doacoes_df.head()

,nome,especie,sexo,idade,porte,extra_info


In [ ]:
registro_animal = []

def registro(nome,especie,sexo,porte,idade,extra_info):
      registro_animal.append (nome)
      registro_animal.append (especie)
      registro_animal.append (sexo)
      registro_animal.append (idade)
      registro_animal.append (porte)
      registro_animal.append (extra_info)

In [ ]:
db_doacoes = []

for count in range(0,12):
  url = rq.get (f'https://portalanimal.campinas.sp.gov.br/adocao?field_especie_adocao_target_id=All&field_sexo_adocao_value=All&field_idade_estimada_adocao_target_id=All&field_porte_tamanho_adocao_target_id=All&page={count}')
  soup = BeautifulSoup (url.content, 'html.parser')

  for animal in soup.find_all('div',attrs={"class":"views-field views-field-title my-2 text-center"}):
      # Cria lista para cada animal
      registro_animal = []

      nome = animal.text
      # Captura da proxima classe onde se encontram as demais informacoes sobre o animal
      aux = animal.next.next.next.text
      especie = aux.split(',')[0]  #especie:  cao, gato, cavalo, passaro, etc
      sexo = aux.split(',')[1]   #sexo:  macho, femea
      porte = aux.split(',')[2]  #porte
      idade = aux.split(',')[3]  #idade
      extra_info = aux.split(',')[4]

      #print(nome,especie,sexo,idade,porte,"\n***********\n")
      registro (nome, especie, sexo, porte, idade, extra_info)
      
      # Acrescenta lista registro_animal com 5 atributos na lista principal db_doacoes
      db_doacoes.append(registro_animal) 

# Acrescenta lista registro_animal com 5 atributos na lista principal db_doacoes
    #db_doacoes.append(registro (nome, especie, sexo, porte, idade, extra_info)) 


In [ ]:
db_doacoes

[['Mimosa', ' Gato', ' Fêmea', ' Senil ( Idoso )', ' Médio', ' Tranquilo\n'],
 ['Soneca', ' Cachorro', ' Macho', ' Adulto', ' Pequeno', ' Carinhoso'],
 ['Draco', ' Gato', ' Macho', ' Adulto', ' Pequeno', ' Agressivo/Feral'],
 ['Moisés', ' Cachorro', ' Macho', ' Senil ( Idoso )', ' Médio', ' Carinhoso'],
 ['Zeca', ' Cachorro', ' Macho', ' Jovem', ' Médio', ' Carinhoso'],
 ['Aninha', ' Cachorro', ' Fêmea', ' Jovem', ' Pequeno', ' Dócil'],
 ['Maria de Nazaré', ' Cachorro', ' Fêmea', ' Jovem', ' Médio', ' Dócil'],
 ['Madalena', ' Cachorro', ' Fêmea', ' Filhote', ' Médio', ' Brincalhão'],
 ['Marcos', ' Cachorro', ' Macho', ' Filhote', ' Médio', ' Carinhoso'],
 ['Lucas', ' Cachorro', ' Macho', ' Filhote', ' Médio', ' Carinhoso'],
 ['Matheus', ' Cachorro', ' Macho', ' Filhote', ' Médio', ' Brincalhão'],
 ['Valentina', ' Gato', ' Fêmea', ' Jovem', ' Pequeno', ' Tranquilo\n'],
 ['Georgia', ' Gato', ' Fêmea', ' Jovem', ' Pequeno', ' Assustado\n'],
 ['Zulu', ' Gato', ' Macho', ' Jovem', ' Médio',

In [ ]:
db_doacoes_df = pd.DataFrame (db_doacoes, columns=['nome','especie','sexo','idade','porte','extra_info'])
db_doacoes_df

,nome,especie,sexo,idade,porte,extra_info
0,Mimosa,Gato,Fêmea,Senil ( Idoso ),Médio,Tranquilo\n
1,Soneca,Cachorro,Macho,Adulto,Pequeno,Carinhoso
2,Draco,Gato,Macho,Adulto,Pequeno,Agressivo/Feral
3,Moisés,Cachorro,Macho,Senil ( Idoso ),Médio,Carinhoso
4,Zeca,Cachorro,Macho,Jovem,Médio,Carinhoso
...,...,...,...,...,...,...
243,CONAN,Gato,Macho,Jovem,Médio,Assustado\n
244,PIRATA,Gato,Macho,Adulto,Médio,Assustado
245,ASLAN,Gato,Macho,Adulto,Médio,Assustado
246,LUCK,Cachorro,Macho,Adulto,Médio,Carinhoso


In [ ]:
## SQL - db_doacoes
## explorar comandos - QUERIES

db_doacoes_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 248 entries, 0 to 247
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   nome        248 non-null    object
 1   especie     248 non-null    object
 2   sexo        248 non-null    object
 3   idade       248 non-null    object
 4   porte       248 non-null    object
 5   extra_info  248 non-null    object
dtypes: object(6)
memory usage: 11.8+ KB


In [ ]:
db_doacoes_df.especie.value_counts()

 Cachorro    127
 Gato        121
Name: especie, dtype: int64

In [ ]:
fig = px.pie (db_doacoes_df, names='especie', title='Distribuição por Espécie',)
fig.show()

In [ ]:
db_doacoes_df.sexo.value_counts()

 Macho    174
 Fêmea     74
Name: sexo, dtype: int64

In [ ]:
fig = px.pie (db_doacoes_df, names='sexo', title='Distribuição por Sexo',)
fig.show()

In [ ]:
db_doacoes_df.describe(include=['object'])

,nome,especie,sexo,idade,porte,extra_info
count,248,248,248,248,248,248
unique,247,2,2,4,3,15
top,MARIO - EM TRATAMENTO!,Cachorro,Macho,Adulto,Médio,Carinhoso
freq,2,127,174,142,143,52


In [ ]:
db = sqlite3.connect('portal_animal_campinas')

In [ ]:
db_doacoes_df.to_sql('doacoes', db, index=False, if_exists='replace')

In [ ]:
db_doacoes_df.to_excel('doacoes.xlsx')

In [ ]:
def executaQuery (query):
  return pd.read_sql(query,db)

In [ ]:
executaQuery('select * from doacoes')

,nome,especie,sexo,idade,porte,extra_info
0,Mimosa,Gato,Fêmea,Senil ( Idoso ),Médio,Tranquilo\n
1,Soneca,Cachorro,Macho,Adulto,Pequeno,Carinhoso
2,Draco,Gato,Macho,Adulto,Pequeno,Agressivo/Feral
3,Moisés,Cachorro,Macho,Senil ( Idoso ),Médio,Carinhoso
4,Zeca,Cachorro,Macho,Jovem,Médio,Carinhoso
...,...,...,...,...,...,...
243,CONAN,Gato,Macho,Jovem,Médio,Assustado\n
244,PIRATA,Gato,Macho,Adulto,Médio,Assustado
245,ASLAN,Gato,Macho,Adulto,Médio,Assustado
246,LUCK,Cachorro,Macho,Adulto,Médio,Carinhoso


In [ ]:
query = '''select nome, idade, extra_info from doacoes 
            where especie LIKE "%Cachorro" 
            AND sexo LIKE "%Fêmea" 
            AND  porte LIKE "%Pequeno" '''
executaQuery(query)

,nome,idade,extra_info
0,Aninha,Jovem,Dócil
1,Loba,Jovem,Assustado\n
2,Flora,Senil ( Idoso ),Carinhoso
3,ISA - EM TRATAMENTO,Adulto,Obediente


In [ ]:
executaQuery('select distinct extra_info from doacoes')

,extra_info
0,Tranquilo\n
1,Carinhoso
2,Agressivo/Feral
3,Dócil
4,Brincalhão
5,Assustado\n
6,Dócil\n
7,Arredio
8,Assustado
9,Obediente
